In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split


In [57]:
import pickle
cleaned_data = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\cleaned_data_Maor.pkl")
X_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_train.pkl")
X_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\X_test.pkl")
y_train = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_train.pkl")
y_test = pd.read_pickle(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\processed\y_test.pkl")
predictions = pd.read_csv(r'C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\predictions\predictions.csv')

In [43]:
raw_data = pd.read_csv(r"C:\Users\maorb\Classes\Classical_ML\YDATA-kaggle-assignment\data\raw\train_dataset_full.csv")

In [32]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365875 entries, 0 to 365874
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   session_id              365167 non-null  float64 
 1   DateTime                365139 non-null  object  
 2   user_id                 365875 non-null  float64 
 3   product                 365799 non-null  object  
 4   campaign_id             365132 non-null  float64 
 5   webpage_id              365132 non-null  float64 
 6   user_group_id           350837 non-null  float64 
 7   gender                  351489 non-null  object  
 8   age_level               351488 non-null  float64 
 9   user_depth              350831 non-null  float64 
 10  city_development_index  266546 non-null  float64 
 11  var_1                   365806 non-null  float64 
 12  is_click                365798 non-null  category
 13  product_category        365235 non-null  float64 
dtypes: c

In [12]:
X_train.isna().sum()    

session_id                  562
DateTime                    589
user_id                       0
product                      54
campaign_id                 575
webpage_id                  590
user_group_id             11989
gender                    11465
age_level                 11460
user_depth                11992
city_development_index    79385
var_1                        47
product_category            499
dtype: int64

In [58]:
def fill_missing_with_mode(df: pd.DataFrame, columns: list):
    """
    Fills missing values in specified columns with their mode.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        columns (list): List of column names to process.

    Returns:
        pd.DataFrame: DataFrame with missing values imputed.
    """
    for column in columns:
        if column in df.columns:
            mode_value = df[column].mode()[0]  # Calculate the mode
            df[column] = df[column].fillna(mode_value)  # Fill missing values with the mode
    return df


In [59]:
#Fill each Na with X_trai
columns_to_fill = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "age_level", "user_depth", "city_development_index", "var_1", "product_category"]

X_train_filled = fill_missing_with_mode(X_train, columns_to_fill)
X_test_filled = fill_missing_with_mode(X_test, columns_to_fill)

In [60]:
X_train_filled.drop(columns = ['session_id', 'DateTime'], inplace = True)
X_test_filled.drop(columns = ['session_id', 'DateTime'], inplace = True)

In [61]:
columns_to_d = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "product_category"]

X_train_filled_d = pd.get_dummies(X_train_filled, columns = columns_to_d)
X_test_filled_d = pd.get_dummies(X_test_filled, columns = columns_to_d)

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_filled_d, y_train)
y_pred = rf.predict(X_test_filled_d)

f1_score(y_test, y_pred)

0.09282202556538839

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
#adjuct the balance of the classes
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_filled_d, y_train)

y_pred_LR = model.predict(X_test_filled_d)
print(f1_score(y_test, y_pred_LR))

0.1450284819524311


In [68]:
from sklearn.metrics import classification_report  
print(classification_report(y_test, y_pred_LR))

              precision    recall  f1-score   support

         0.0       0.94      0.57      0.71     68166
         1.0       0.08      0.54      0.15      4994

    accuracy                           0.57     73160
   macro avg       0.51      0.55      0.43     73160
weighted avg       0.89      0.57      0.67     73160



**Baseline? :()**

In [7]:
cleaned_data['is_click'].value_counts()[0]/cleaned_data['is_click'].value_counts().sum()

0.9323260378678943

## Naive preprocessing (not my class)

In [52]:
data2= raw_data.drop(columns = ['session_id', 'DateTime', 'user_id','product_category_2'])
data2.dropna(inplace = True) # Just close your eyes and drop the rows with missing values

In [72]:
X_2 = data2.drop(columns = ['is_click'])
y_2 = data2['is_click']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

In [74]:
columns_to_d2 = ["product", "campaign_id", "webpage_id", "user_group_id", "gender", "product_category_1"]

X_train_2_d = pd.get_dummies(X_train_2, columns = columns_to_d2)
X_test_2_d = pd.get_dummies(X_test_2, columns = columns_to_d2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(n_estimators=100, random_state=42,class_weight='balanced')
rf.fit(X_train_2_d, y_train_2)
y_pred_RF = rf.predict(X_test_2_d)



In [79]:
f1_score(y_test_2, y_pred_RF)

0.14298031865042174

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_2_d, y_train_2)

y_pred_LR = model.predict(X_test_2_d)
print(f1_score(y_test_2, y_pred_LR))

0.14430736693690518
